# GOAL
- To identify the 1/50 of the online sales that should be put at the back of the store to facilitate picking

## PROBLEM STRCTURE
- build basic table: store_cd, mtrl_cd, mtrl_name, item_flow, single_line_order, count_lines
- create function to calculate pdm and cum_pdm based on the desired filters: item_flow, single_line_order 

# PACKAGES

In [14]:
from google.cloud import bigquery
import pandas as pd
from datetime import datetime
import pandas_profiling

# PARAMETERS

In [5]:
client = bigquery.Client(project='brlm-web-data')
project_name = client.project
print("Client created using default project: {}".format(project_name))

Client created using default project: brlm-web-data


# FUNCTIONS

In [232]:
def identify_single_line_tiket(df,group_cols=['CSHR_GRP_NR','TKT_NR'],count_col='MTRL_CD'):
    """ (df, list(str), str) -> df
    Append a boolean column named SINGLE_LINE to df. 
    SINGLE_LINE is True if only one line is found when grouping the columns group_columns.
    It is False otherwise
    """
    df_wip = df.copy()
    df_wip['SINGLE_LINE']=df_wip.groupby(by=group_cols)[count_col].transform('count').apply(lambda x: True if x==1 else False)
    
    return df_wip

In [233]:
def count_lines(df,group_cols=['CSHR_GRP_NR','MTRL_CD','MTRL_TEXT','ITEM_OUT_FLOW','CHNL_SLS','SINGLE_LINE'],count_col='TKT_NR'):
    """ (df, list(str), str) -> df
    Create an aggregated DF and include a new column COUNT_LINES.
    COUNT_LINES stores the number of occurences of count_columns in each group formed by grou_columns.
    
    It is important to include in group_cols all dimensions that could be used in the future.
    """
    return df.groupby(by=group_cols, as_index=False)[count_col].count().rename(columns={count_col:'COUNT_LINES'})

In [234]:
def calculate_pdm(df,group_cols,sum_col='COUNT_LINES', mtrl_cd='MTRL_CD',mtrl_text='MTRL_TEXT'):
    """ (df, list(str), str) -> df
    Calculate and append 3 columns to df:TOT_LINES, PDM and CUM_PDM.
    TOT_LINES is the sum of sum_col when data is grouped by group_cols.
    PDM is the ratio between the value of sum_col each row and TOT_LINES.
    CUM_PDM is the cumulative sum PDM each material in a group.
    
    group_cols should contain only the columns of interest.
    """

    df_wip=df.copy()
    df_wip=df_wip.groupby(by=group_cols + [mtrl_cd,mtrl_text],as_index=False)[sum_col].sum()
    df_wip['TOT_LINES']=df_wip.groupby(by=group_cols)[sum_col].transform('sum')
    df_wip['PDM']=df_wip['COUNT_LINES']/df_wip['TOT_LINES']
    df_wip=df_wip.sort_values(by=group_cols+ ['PDM'], ascending=False,ignore_index=True)
    df_wip['CUM_PDM']=df_wip.groupby(by=group_cols)['PDM'].transform('cumsum')
    
    return df_wip

# DATA WRANGLING
- Get sample data from FT_SALES (plnt_cd = 58, sales_dt in the last week, channel=e-commerce)
- Understand data structure
- create function to calculate the representativeness of each product in the total sales
- create function to calculate the cumulated representativeness of each product in the total sales

- CSHR_OPERTN_TYP: SALES, REFUND ETC. 1, 96, 101, 103, 105 are positive. 97,100,98,104,102 are negative. Probably I should use only the positive ones.
- CHNL_SLS: 1 - Loja, 2- Televendas, 3-e-commerce. To use only 2 and 3.
- ITEM_OUT_FLOW:  'ED' 1, 'RI' 2, 'RE' 3, 'CXA' 4 ELSE NULL. To be ignored for the moment. If the sales came from e-com, it doens not matter how it was delivered to the client.

## Get sample data from FT_SLS

In [56]:
query = """
SELECT
    PLNT_CD,
    SLS_DT,
    OPER_TYP,
    CSHR_GRP_NR,
    CSHR_OPERTN_TYP,
    MTRL_CD,
    SUPLY_TYP_MTRL,
    MTRL_QTY,
    GRS_SLS_AMT,
    CHNL_SLS,
    ITEM_OUT_FLOW,
    TKT_NR,
    SLS_ORD_NR
FROM
    `brlm-web-data.dtmsls.FT_SLS`
WHERE
    CSHR_GRP_NR = 58
    AND SLS_DT BETWEEN '2021-03-15' AND '2021-03-07'
    AND CHNL_SLS > 1
    AND CSHR_OPERTN_TYP IN (1, 96, 101, 103, 105)
"""
df = client.query(query).to_dataframe()
df_bkp=df.copy()

## Understand data structure on FT_SLS

In [15]:
profile=pandas_profiling.ProfileReport(df)
profile

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

- OPER_TYPE is missing
- CSHR_OPER_TYP = 101
- 6 more TKT_NR than SLS_ORD_NR

## Create pdm function
- target df = plant, mtrl, tot_qty, pdm, cum_pdm

In [92]:
df.head(1)

,PLNT_CD,SLS_DT,OPER_TYP,CSHR_GRP_NR,CSHR_OPERTN_TYP,MTRL_CD,SUPLY_TYP_MTRL,MTRL_QTY,GRS_SLS_AMT,CHNL_SLS,ITEM_OUT_FLOW,TKT_NR,SLS_ORD_NR,COUNT_ITEM_TKT
0,58,2021-03-07,NaN,58,101,90823194,1,2.0,259.8,3,3,2004307874,10669137,1


In [93]:
# create df with single line tikets 
by_columns=['CSHR_GRP_NR','TKT_NR']
df['COUNT_ITEM_TKT']=df.groupby(by=by_columns)['MTRL_CD'].transform('count')
df.sample()

print('all tikets',df.shape)
df_s=df[df['COUNT_ITEM_TKT']==1].drop(columns='COUNT_ITEM_TKT')
print('only single line tikets', df_s.shape)
df_s.head(1)

all tikets (1946, 14)
only single line tikets (700, 13)


,PLNT_CD,SLS_DT,OPER_TYP,CSHR_GRP_NR,CSHR_OPERTN_TYP,MTRL_CD,SUPLY_TYP_MTRL,MTRL_QTY,GRS_SLS_AMT,CHNL_SLS,ITEM_OUT_FLOW,TKT_NR,SLS_ORD_NR
0,58,2021-03-07,NaN,58,101,90823194,1,2.0,259.8,3,3,2004307874,10669137


In [94]:
# calculate pdm by mtrl_cd
in_columns=['CSHR_GRP_NR','MTRL_CD']
by_columns=['CSHR_GRP_NR','MTRL_CD']
df_s_pdm=df_s.groupby(by=by_columns,as_index=False)['MTRL_QTY'].count().rename(columns={'MTRL_QTY':'LINES_QTY'})
df_s_pdm.sample(1)

,CSHR_GRP_NR,MTRL_CD,LINES_QTY
446,58,90596331,2


In [95]:
df_s_pdm['TOT_LINES_PLNT']=df_s_pdm.groupby(by='CSHR_GRP_NR')['LINES_QTY'].transform('sum')
df_s_pdm.head(1)

,CSHR_GRP_NR,MTRL_CD,LINES_QTY,TOT_LINES_PLNT
0,58,85130871,1,700


In [96]:
df_s_pdm['PDM']=df_s_pdm['LINES_QTY']/df_s_pdm['TOT_LINES_PLNT']
df_s_pdm.head(1)

,CSHR_GRP_NR,MTRL_CD,LINES_QTY,TOT_LINES_PLNT,PDM
0,58,85130871,1,700,0.001429


In [97]:
df_s_pdm=df_s_pdm.sort_values(by=['CSHR_GRP_NR','PDM'], ascending=False,ignore_index=True)
df_s_pdm.head(10)

,CSHR_GRP_NR,MTRL_CD,LINES_QTY,TOT_LINES_PLNT,PDM
0,58,90601336,88,700,0.125714
1,58,89172342,15,700,0.021429
2,58,89840632,8,700,0.011429
3,58,89903660,8,700,0.011429
4,58,85987230,7,700,0.010000
5,58,89866623,6,700,0.008571
6,58,90883534,5,700,0.007143
7,58,87163195,4,700,0.005714
8,58,89550615,4,700,0.005714
9,58,89643981,4,700,0.005714


In [98]:
df_s_pdm['CUM_PDM']=df_s_pdm.groupby(by=['CSHR_GRP_NR'])['PDM'].transform('cumsum')
df_s_pdm.head(10)

,CSHR_GRP_NR,MTRL_CD,LINES_QTY,TOT_LINES_PLNT,PDM,CUM_PDM
0,58,90601336,88,700,0.125714,0.125714
1,58,89172342,15,700,0.021429,0.147143
2,58,89840632,8,700,0.011429,0.158571
3,58,89903660,8,700,0.011429,0.170000
4,58,85987230,7,700,0.010000,0.180000
5,58,89866623,6,700,0.008571,0.188571
6,58,90883534,5,700,0.007143,0.195714
7,58,87163195,4,700,0.005714,0.201429
8,58,89550615,4,700,0.005714,0.207143
9,58,89643981,4,700,0.005714,0.212857


In [100]:
df_s_pdm[df_s_pdm['PDM']>0.05]

,CSHR_GRP_NR,MTRL_CD,LINES_QTY,TOT_LINES_PLNT,PDM,CUM_PDM
0,58,90601336,88,700,0.125714,0.125714


## Get only the necessary data from FT_SLS AND MTRL_CD

In [169]:
query = """
SELECT
    CSHR_GRP_NR,
    FT_SLS.MTRL_CD,
    DIM_MTRL.MTRL_TEXT,
    ITEM_OUT_FLOW,
    TKT_NR
FROM
    dtmsls.FT_SLS, dtmpro.DIM_MTRL
WHERE
    DIM_MTRL.MTRL_CD = CAST(FT_SLS.MTRL_CD AS STRING)
    AND CSHR_GRP_NR = 58
    AND SLS_DT BETWEEN '2021-02-15' AND '2021-03-12'
    AND ITEM_OUT_FLOW IN (1,3)
    AND CSHR_OPERTN_TYP IN (1, 96, 101, 103, 105)
"""
df = client.query(query).to_dataframe()
df_bkp=df.copy()

In [189]:
df.sample(1)

,CSHR_GRP_NR,MTRL_CD,MTRL_TEXT,ITEM_OUT_FLOW,TKT_NR
6064,58,90539736,MAX-AR CONFORT MBOR BCO 60X60,1,2004061465


## Build basic table: store_cd, mtrl_cd, mtrl_name, item_flow, single_line_order, count_lines

In [190]:
df=df_bkp.copy()

In [191]:
def identify_single_line_tiket(df,group_cols=['CSHR_GRP_NR','TKT_NR'],count_col='MTRL_CD'):
    """ (df, list(str), str) -> df
    Append a boolean column named SINGLE_LINE to df. 
    SINGLE_LINE is True if only one line is found when grouping the columns group_columns.
    It is False otherwise
    """
    df_wip = df.copy()
    df_wip['SINGLE_LINE']=df_wip.groupby(by=group_cols)[count_col].transform('count').apply(lambda x: True if x==1 else False)
    
    return df_wip
    
df_wip=identify_single_line_tiket(df,['CSHR_GRP_NR','TKT_NR'],'MTRL_CD')
df_wip.sample()

,CSHR_GRP_NR,MTRL_CD,MTRL_TEXT,ITEM_OUT_FLOW,TKT_NR,SINGLE_LINE
3348,58,89792185,Caixa Acopl Eco/Azale/Acess Br 3/6L Roca,1,2004157910,False


In [192]:
def count_lines(df,group_cols=['CSHR_GRP_NR','MTRL_CD','ITEM_OUT_FLOW','SINGLE_LINE'],count_col='TKT_NR'):
    """ (df, list(str), str) -> df
    Create an aggregated DF and include a new column COUNT_LINES.
    COUNT_LINES stores the number of occurences of count_columns in each group formed by grou_columns.
    
    It is important to include in group_cols all dimensions that could be used in the future.
    """
    return df.groupby(by=group_cols, as_index=False)[count_col].count().rename(columns={count_col:'COUNT_LINES'})

df_wip=count_lines(df_wip)
df_wip.sample()

,CSHR_GRP_NR,MTRL_CD,ITEM_OUT_FLOW,SINGLE_LINE,COUNT_LINES
5821,58,90437116,1,False,2


In [193]:
df_wip=count_lines(identify_single_line_tiket(df=df))
df_wip.sample()

,CSHR_GRP_NR,MTRL_CD,ITEM_OUT_FLOW,SINGLE_LINE,COUNT_LINES
665,58,86839683,1,True,2


## Create function to calculate pdm and cum_pdm based on the desired filters: item_flow, single_line_order  

In [197]:
def calculate_pdm(df,group_cols,sum_col='COUNT_LINES', mtrl_cd='MTRL_CD'):
    """ (df, list(str), str) -> df
    Calculate and append 3 columns to df:TOT_LINES, PDM and CUM_PDM.
    TOT_LINES is the sum of sum_col when data is grouped by group_cols.
    PDM is the ratio between the value of sum_col each row and TOT_LINES.
    CUM_PDM is the cumulative sum PDM each material in a group.
    
    group_cols should contain only the columns of interest.
    """

    df_wip=df.copy()
    df_wip=df_wip.groupby(by=group_cols + [mtrl_cd],as_index=False)[sum_col].sum()
    df_wip['TOT_LINES']=df_wip.groupby(by=group_cols)[sum_col].transform('sum')
    df_wip['PDM']=df_wip['COUNT_LINES']/df_wip['TOT_LINES']
    df_wip=df_wip.sort_values(by=group_cols+ ['PDM'], ascending=False,ignore_index=True)
    df_wip['CUM_PDM']=df_wip.groupby(by=group_cols)['PDM'].transform('cumsum')
    
    return df_wip

by_cols_pdm=['CSHR_GRP_NR','ITEM_OUT_FLOW','SINGLE_LINE']
calculate_pdm(df=df_wip,group_cols=by_cols_pdm)

,CSHR_GRP_NR,ITEM_OUT_FLOW,SINGLE_LINE,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,3,True,89368944,83,1904,0.043592,0.043592
1,58,3,True,89851566,28,1904,0.014706,0.058298
2,58,3,True,89296165,24,1904,0.012605,0.070903
3,58,3,True,89436284,24,1904,0.012605,0.083508
4,58,3,True,89870480,20,1904,0.010504,0.094013
...,...,...,...,...,...,...,...,...
6763,58,1,False,90997501,1,5908,0.000169,0.999323
6764,58,1,False,90997550,1,5908,0.000169,0.999492
6765,58,1,False,90997564,1,5908,0.000169,0.999661
6766,58,1,False,90997571,1,5908,0.000169,0.999831


In [198]:
by_cols_pdm=['CSHR_GRP_NR','ITEM_OUT_FLOW','SINGLE_LINE']

df_wip=calculate_pdm(
    count_lines(
        identify_single_line_tiket(df=df)),
    group_cols=by_cols_pdm)
df_wip.describe()

,CSHR_GRP_NR,ITEM_OUT_FLOW,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
count,6768.0,6768.000000,6.768000e+03,6768.000000,6768.000000,6768.000000,6768.000000
mean,58.0,1.684397,8.909239e+07,1.917553,3959.969710,0.000591,0.708688
std,0.0,0.948961,1.446285e+06,4.219344,1764.382505,0.001512,0.201979
min,58.0,1.000000,8.500921e+07,1.000000,1904.000000,0.000169,0.017773
25%,58.0,1.000000,8.826687e+07,1.000000,2345.000000,0.000169,0.584644
50%,58.0,1.000000,8.949981e+07,1.000000,2821.000000,0.000354,0.739450
75%,58.0,3.000000,9.000405e+07,2.000000,5908.000000,0.000525,0.869770
max,58.0,3.000000,9.100277e+07,187.000000,5908.000000,0.079744,1.000000


In [199]:
df_wip[df_wip['PDM']>0.02]

,CSHR_GRP_NR,ITEM_OUT_FLOW,SINGLE_LINE,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,3,True,89368944,83,1904,0.043592,0.043592
982,58,3,False,89368944,84,2821,0.029777,0.029777
983,58,3,False,89851566,72,2821,0.025523,0.055300
2316,58,1,True,90601336,187,2345,0.079744,0.079744


### by_cols_pdm=['CSHR_GRP_NR','ITEM_OUT_FLOW','SINGLE_LINE']

In [203]:
by_cols_pdm=['CSHR_GRP_NR','ITEM_OUT_FLOW','SINGLE_LINE']

df_wip=calculate_pdm(
    count_lines(identify_single_line_tiket(df=df)),
    group_cols=by_cols_pdm)

df_wip[df_wip['PDM']>0.01]

,CSHR_GRP_NR,ITEM_OUT_FLOW,SINGLE_LINE,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,3,True,89368944,83,1904,0.043592,0.043592
1,58,3,True,89851566,28,1904,0.014706,0.058298
2,58,3,True,89296165,24,1904,0.012605,0.070903
3,58,3,True,89436284,24,1904,0.012605,0.083508
4,58,3,True,89870480,20,1904,0.010504,0.094013
982,58,3,False,89368944,84,2821,0.029777,0.029777
983,58,3,False,89851566,72,2821,0.025523,0.055300
984,58,3,False,89296165,53,2821,0.018788,0.074087
985,58,3,False,89296200,40,2821,0.014179,0.088267
986,58,3,False,90185102,38,2821,0.013470,0.101737


### by_cols_pdm=['CSHR_GRP_NR','SINGLE_LINE']

In [204]:
by_cols_pdm=['CSHR_GRP_NR','SINGLE_LINE']

df_wip1=calculate_pdm(
    count_lines(identify_single_line_tiket(df=df)),
    group_cols=by_cols_pdm)

df_wip1[(df_wip1['PDM']>0.01)]

,CSHR_GRP_NR,SINGLE_LINE,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,True,90601336,190,4249,0.044716,0.044716
1,58,True,89368944,90,4249,0.021181,0.065898
2,58,True,89840632,47,4249,0.011061,0.076959
2120,58,False,89368944,189,8729,0.021652,0.021652
2121,58,False,89851566,144,8729,0.016497,0.038149
2122,58,False,89296165,88,8729,0.010081,0.048230


### by_cols_pdm=['CSHR_GRP_NR']

In [205]:
by_cols_pdm=['CSHR_GRP_NR']

df_wip1=calculate_pdm(
    count_lines(identify_single_line_tiket(df=df)),group_cols=by_cols_pdm)

df_wip1[(df_wip1['PDM']>0.01)]

,CSHR_GRP_NR,MTRL_CD,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,89368944,279,12978,0.021498,0.021498
1,58,90601336,218,12978,0.016798,0.038296
2,58,89851566,175,12978,0.013484,0.051780


# RUN

## Run for all stores and last 15 days

In [228]:
query = """
SELECT
    CSHR_GRP_NR,
    FT_SLS.MTRL_CD,
    DIM_MTRL.MTRL_TEXT,
    ITEM_OUT_FLOW,
    TKT_NR,
    CASE CHNL_SLS
        WHEN 1 THEN 'OFFLINE'
        WHEN 2 THEN 'ONLINE'
        WHEN 3 THEN 'ONLINE'
    END AS CHNL_SLS
FROM
    dtmsls.FT_SLS, dtmpro.DIM_MTRL
WHERE
    DIM_MTRL.MTRL_CD = CAST(FT_SLS.MTRL_CD AS STRING)
    AND CSHR_GRP_NR = 58
    AND SLS_DT BETWEEN '2021-02-25' AND '2021-03-11'
    AND ITEM_OUT_FLOW IN (1,3)
    AND CSHR_OPERTN_TYP IN (1, 96, 101, 103, 105)
"""
df = client.query(query).to_dataframe()

In [231]:
df.sample()

,CSHR_GRP_NR,MTRL_CD,MTRL_TEXT,ITEM_OUT_FLOW,TKT_NR,CHNL_SLS
2991,58,89388684,ARG AC-III MULTIUSO BCO 20KG AXTON,1,2004257451,OFFLINE


In [236]:
by_cols_pdm=['CSHR_GRP_NR']
df_store=calculate_pdm(
    count_lines(identify_single_line_tiket(df=df)),group_cols=by_cols_pdm)
df_store=df_store[(df_store['PDM']>0.01)]
df_store
#df_store.to_csv(datetime.today().strftime('%Y%m%d') + '_Produtos com maior pdm by '+ str(by_cols_pdm) + '.csv',
#               sep=';', index=False,decimal=',')

,CSHR_GRP_NR,MTRL_CD,MTRL_TEXT,COUNT_LINES,TOT_LINES,PDM,CUM_PDM
0,58,90601336,MASCARA PFF2 PRO AGRO Delta,199,7750,0.025677,0.025677
1,58,89368944,CIMENTO CPII 32 TODAS AS OBRA 50KG VOT,157,7750,0.020258,0.045935
2,58,89851566,Areia Media lavada 20 kg,111,7750,0.014323,0.060258


In [240]:
scope = [['CSHR_GRP_NR'],
         ['CSHR_GRP_NR','CHNL_SLS'],['CSHR_GRP_NR','SINGLE_LINE'],['CSHR_GRP_NR','ITEM_OUT_FLOW'],
         ['CSHR_GRP_NR','ITEM_OUT_FLOW','SINGLE_LINE'],['CSHR_GRP_NR','CHNL_SLS','SINGLE_LINE'],['CSHR_GRP_NR','CHNL_SLS','ITEM_OUT_FLOW'],
         ['CSHR_GRP_NR','CHNL_SLS','ITEM_OUT_FLOW','SINGLE_LINE']]

In [239]:
for item in scope:
    df_result=calculate_pdm(
    count_lines(identify_single_line_tiket(df=df)),group_cols=item)

    df_result=df_result[(df_result['PDM']>0.01)]
    df_result.to_csv(datetime.today().strftime('%Y%m%d') + '_Produtos com maior pdm by '+ str(item) + '.csv',
               sep=';', index=False,decimal=',')